In [1]:
import requests 
import re as regex
import json
import pprint as pp # pretty print

import aries_cloudagent.wallet.crypto as crypto
import base64
import base58
import uuid
import jupyter_util as util

## Enter Invite Url:

In [2]:
agent1 = {}
plugin = {}
endpoint = "http://localhost:8150/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent1['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent1['url'])

http://agent1.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiYjg2OWI0NjAtMzc1ZS00MTQ0LWEyOGUtZGFjN2NiMjRhM2E3IiwgImxhYmVsIjogIk1haW4gKGFkbWluKSIsICJzZXJ2aWNlRW5kcG9pbnQiOiAiaHR0cDovL2FnZW50MS5sb2NhbGhvc3QiLCAicmVjaXBpZW50S2V5cyI6IFsiNW9HZDRUWlUxR2ZkZUs4OVJVMnhMcHltR3JzUEU2SHZwS1hVTkE0ZnFpRHAiXX0=


In [3]:
# Process invite url, delete white spaces
agent1['url'] = agent1['url'].replace(" ", "")
# Regex(substitution) to extract only the invite string from url
agent1['invite_string_b64'] = regex.sub(
           r".*(c\_i\=)", 
           r"", 
           agent1['url'])
# Decoding invite string using base64 decoder
agent1['invite_string'] = base64.b64decode(agent1['invite_string_b64'])
# Converting our invite json string into a dictionary 
agent1['invite'] = json.loads(agent1['invite_string'])

print("Decoded invitation:\n")
pp.pprint(agent1['invite'])

Decoded invitation:

{'@id': 'b869b460-375e-4144-a28e-dac7cb24a3a7',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
 'label': 'Main (admin)',
 'recipientKeys': ['5oGd4TZU1GfdeK89RU2xLpymGrsPE6HvpKXUNA4fqiDp'],
 'serviceEndpoint': 'http://agent1.localhost'}


In [4]:
### Generating a did key using aries crypto module
# Create a key pair with random seed 
# Order: 0 - public key, 1 - secret / pritvate key
plugin['keypair'] = crypto.create_keypair()
# it seems to me that did is an encoded sub string of public key
# or maybe just some arbitrary random number
did = plugin['keypair'][0][:16]
# final version of our did
plugin['did'] = base58.b58encode(did).decode("ascii")
# encoding keys to base58
plugin['public_key_b58'] = \
    base58.b58encode(plugin['keypair'][0]).decode("ascii")
plugin['private_key_b58'] = \
    base58.b58encode(plugin['keypair'][1]).decode("ascii")

# print(base58.b58encode(did))
# print(plugin['did'])
print("Private key: \n", plugin['private_key_b58'])
print("\nPublic key: \n", plugin['public_key_b58'])

Private key: 
 f84DWijNoviU8iBRbKppPFP9tb1c3DJb9n7K9PPH5HtmRPAHutynjVSn2cvDwkibx2vgMzp7aUGGQdsYcFXZwbN

Public key: 
 FMHjyQLYY9szZbT65VQ3aPAFztaG66fA83BxyeZ2dv9E


**Sending a connection request to acapy**

In [5]:
uniqueId = str(uuid.uuid4())
# print(uniqueId)

# our request body
message = {
        "@id":  uniqueId,
        "~transport": {
          "return_route": "all"
        },
        "@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/request",
        "label": "Plugin",
        "connection": {
          "DID": plugin['did'],
          "DIDDoc": {
            "@context": "https://w3id.org/did/v1",
            "id": plugin['did'],
            "publicKey": [{
              "id": plugin['did'] + "#keys-1",
              "type": "Ed25519VerificationKey2018",
              "controller": plugin['did'],
              "publicKeyBase58": plugin['public_key_b58']
            }],
            "service": [{
              "id": plugin['did'] + ";indy",
              "type": "IndyAgent",
              "recipientKeys": plugin['public_key_b58'],
              "serviceEndpoint": ""
            }]
          }
        }
      }

**Encoding/packing the request**

In [6]:
# encoding it with aries crypto function using the key that was
# given to us by aca-py in recipientKeys
decodedAcapyKey = base58.b58decode(agent1['invite']['recipientKeys'][0])
ourPrivateKey = plugin['keypair'][1]
# print(plugin['did'])
encodedMessage = \
    crypto.encode_pack_message(json.dumps(message), [decodedAcapyKey], ourPrivateKey)

encodedMessage = encodedMessage.decode("ascii")

In [7]:
connectionRequestResponse = requests.post(agent1['invite']['serviceEndpoint'], data=encodedMessage)
assert connectionRequestResponse.text != "", "invalid response from acapy"

**Response from aca-py**

In [8]:
   
connectionRequestResponseUnpacked = \
    util.unpackMessage(connectionRequestResponse.text, plugin['keypair'][1])

connectionRequestResponseDict = json.loads(connectionRequestResponseUnpacked[0])

print("Decoded acapy response: \n")
pp.pprint(connectionRequestResponseUnpacked)

Decoded acapy response: 

('{"@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/response", '
 '"@id": "3b7cee7b-1900-4500-aa70-c1bab8f74c5c", "~thread": {"thid": '
 '"c766fce3-e886-4928-95d3-3c8aaaf5651e"}, "connection~sig": {"@type": '
 '"did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/signature/1.0/ed25519Sha512_single", '
 '"signature": '
 '"6dXlVT_-BqiqXn4jIzlwv3HQ4vpjvv0VJCTCuIudwvoY9FDmn98dCRfG88akX5NNdb2GTlbe_8oanLlnbLvGBg==", '
 '"sig_data": '
 '"AAAAAF7DrK17IkRJRCI6ICJXVFNuUU1VZFFqZHN4WWtLVUJUY1lKIiwgIkRJRERvYyI6IHsiQGNvbnRleHQiOiAiaHR0cHM6Ly93M2lkLm9yZy9kaWQvdjEiLCAiaWQiOiAiZGlkOnNvdjpXVFNuUU1VZFFqZHN4WWtLVUJUY1lKIiwgInB1YmxpY0tleSI6IFt7ImlkIjogImRpZDpzb3Y6V1RTblFNVWRRamRzeFlrS1VCVGNZSiMxIiwgInR5cGUiOiAiRWQyNTUxOVZlcmlmaWNhdGlvbktleTIwMTgiLCAiY29udHJvbGxlciI6ICJkaWQ6c292OldUU25RTVVkUWpkc3hZa0tVQlRjWUoiLCAicHVibGljS2V5QmFzZTU4IjogIkg0QUpOeHlCSGRtZ0VOUXNNRTZkaFhTMzQ1TkxWcWN2dlBNN1F0RjJYM2I1In1dLCAiYXV0aGVudGljYXRpb24iOiBbeyJ0eXBlIjogIkVkMjU1MTlTaWduYXR1cmVBdXRoZW50aWNhdGlvbjIwMT

**Unpacking connection data embedded in the response**

In [9]:
sig_data_raw = connectionRequestResponseDict['connection~sig']['sig_data']
#  (this is a hack)replacing artifacts that sometimes happen
sig_data_raw = sig_data_raw.replace("-", "1")
sig_data_raw = sig_data_raw.replace("_", "1")

sig_data_raw = base64.b64decode(sig_data_raw)
# avoid first 8 characters as they are a time signature
sig_data_raw = sig_data_raw[8:]
sig_data_raw = sig_data_raw.decode('ascii')

sig_data = json.loads(sig_data_raw)
pp.pprint(sig_data)

{'DID': 'WTSnQMUdQjdsxYkKUBTcYJ',
 'DIDDoc': {'@context': 'https://w3id.org/did/v1',
            'authentication': [{'publicKey': 'did:sov:WTSnQMUdQjdsxYkKUBTcYJ#1',
                                'type': 'Ed25519SignatureAuthentication2018'}],
            'id': 'did:sov:WTSnQMUdQjdsxYkKUBTcYJ',
            'publicKey': [{'controller': 'did:sov:WTSnQMUdQjdsxYkKUBTcYJ',
                           'id': 'did:sov:WTSnQMUdQjdsxYkKUBTcYJ#1',
                           'publicKeyBase58': 'H4AJNxyBHdmgENQsME6dhXS345NLVqcvvPM7QtF2X3b5',
                           'type': 'Ed25519VerificationKey2018'}],
            'service': [{'id': 'did:sov:WTSnQMUdQjdsxYkKUBTcYJ;indy',
                         'priority': 0,
                         'recipientKeys': ['H4AJNxyBHdmgENQsME6dhXS345NLVqcvvPM7QtF2X3b5'],
                         'serviceEndpoint': 'http://agent1.localhost',
                         'type': 'IndyAgent'}]}}


**Adding the connection to connection list**

In [10]:
connections = [
    {
        "label": agent1['invite']['label'],
        "DIDDoc": sig_data['DIDDoc'],
        "myKey": plugin['keypair'][1]
    }
]

pp.pprint(connections)

[{'DIDDoc': {'@context': 'https://w3id.org/did/v1',
             'authentication': [{'publicKey': 'did:sov:WTSnQMUdQjdsxYkKUBTcYJ#1',
                                 'type': 'Ed25519SignatureAuthentication2018'}],
             'id': 'did:sov:WTSnQMUdQjdsxYkKUBTcYJ',
             'publicKey': [{'controller': 'did:sov:WTSnQMUdQjdsxYkKUBTcYJ',
                            'id': 'did:sov:WTSnQMUdQjdsxYkKUBTcYJ#1',
                            'publicKeyBase58': 'H4AJNxyBHdmgENQsME6dhXS345NLVqcvvPM7QtF2X3b5',
                            'type': 'Ed25519VerificationKey2018'}],
             'service': [{'id': 'did:sov:WTSnQMUdQjdsxYkKUBTcYJ;indy',
                          'priority': 0,
                          'recipientKeys': ['H4AJNxyBHdmgENQsME6dhXS345NLVqcvvPM7QtF2X3b5'],
                          'serviceEndpoint': 'http://agent1.localhost',
                          'type': 'IndyAgent'}]},
  'label': 'Main (admin)',
  'myKey': b' \xdf\xe9\xb9K\xec\x82~!j\xac\xac\xfb\x93\xda\xf7\xa9\x1

### FEATURE DISCOVERY

In [11]:
message = util.buildMessage(
    "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0/query",
    query="*"
)
message = util.packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=message)
# print(response.text)

responseDecoded = util.unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': 'aca0ce17-2725-400e-9713-69780259c073',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0/disclose',
 'protocols': [{'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-static-connections/0.1'},
               {'pid': 'https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-payments/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-issuer/0.1'},
               {'pid': 'https://didcomm.org/routing/1.0'},
               {'pid': 'https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-basicmessage/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/trust_ping/1.0'},
               {'pid': 'https://didcomm.org/connections/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/routing/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0'},
               {'pid': 'https://didcomm.or

## Dids

In [12]:
message = util.buildMessage("did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/get-list-dids")
encodedMessage = util.packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = util.unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': '07308c74-fbda-441d-817b-621f3a4cd5d2',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/list-dids',
 'result': [{'did': 'WTSnQMUdQjdsxYkKUBTcYJ',
             'verkey': 'H4AJNxyBHdmgENQsME6dhXS345NLVqcvvPM7QtF2X3b5'},
            {'did': 'WjVuoNs4Zc1mK6vXPfqnj5',
             'verkey': 'HCuoA2ewtHWQ297KgJG22VrexYmEbZRCi4aqPkE2cEt9'},
            {'did': 'CcPNApHoaNaxHtrDJx3Fcu',
             'verkey': '7L23hAKHoiLQ4KyaTEkpzi5eoLXt3cLuteFYp6CzDxXk'},
            {'did': 'AH7bQLKyijZkniHvRoLAzu',
             'verkey': '64HqNtZaju4Y5Vd4ZzzBZ4apZqe69gc6AtF9zHr5S9dr'},
            {'did': 'TUNT4Z7kKeYquhmeCZjg8F',
             'verkey': 'FRq4sMiB64yErvd4Rs7k4dmKxzBBVkbFh4Rt6QGmXmbZ'},
            {'did': 'WZndzdvhBc6vXp8hFPrCdL',
             'verkey': 'H7co14Vem8cv3BQ4GLgu3yutTHRRikXfUFofspX9vpDd'},
            {'did': 'qREBhbMGwvNYXYVehH82e',
             'verkey': 'TPabtvsKvYvrGPPohaD6PGy3yn4MxmQbng9BWSgDimx'},
            {'did': '8vxj6PkgTnDZ2bBkyHsxga',
        

## Public did

In [13]:
message = util.buildMessage("did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/get-public-did")
encodedMessage = util.packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = util.unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': '2577870e-7ef1-4978-a0f1-4edddb357ea4',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/did',
 '~thread': {'thid': 'ed7c2969-38c9-465b-ac2e-565bb778a669'}}


In [14]:
message = util.buildMessage(
    "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/schema",
    payload="a very cool message"
)

encodedMessage = util.packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = util.unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': '736c5003-1cd5-4c62-abd0-20b10d282eeb',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/schema',
 'payload': 'a very cool message',
 '~thread': {'thid': '3544ac98-a445-4cc6-85f1-8180c90b89fc'}}


In [16]:
message = util.buildMessage(
    "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/search_schema",
    type="schema"
)

encodedMessage = util.packMessage(message, connections[0])

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)

responseDecoded = util.unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': 'fa527ff4-541b-47d3-b2da-3f8f6ee70631',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/search_schema',
 'type': 'schema'}
